## Notebook settings

In [1]:
%load_ext autoreload
%autoreload 2

# %autosave 10

#%load_ext lab_black

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join("..")))

## Imports

In [204]:
from lfp_analysis.data import *
from lfp_analysis.process import *

from lfp_analysis.resnet1d import *
from lfp_analysis.svm import *

from fastai.vision import *
import torch.nn.functional as F
from torchvision.transforms import ToPILImage, ToTensor

In [205]:
import numpy as np
import pandas as pd
import h5py

from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib widget

In [206]:
ll = list(range(10))
ll

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [209]:
x = np.arange(51)
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [210]:
frame(x,5,5)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49]])

In [161]:
import math
def frame(array,win_len, hop_len, pad = True):
    nrows = ((array.size-win_len)//hop_len)+1
    
    if pad:
        n_reps = math.ceil(1 - (array.size-win_len)/hop_len % 1)
        print(n_reps)
    
    nrows = ((array.size-win_len)//hop_len)+1
    print(nrows)
    n = array.strides[0]
    return np.lib.stride_tricks.as_strided(array, shape=(nrows,win_len), strides=(hop_len*n,n))

In [ ]:
frame(x,5,1)

In [ ]:
sig

In [176]:
pad = True
array = np.arange(50)
win_len = 5
hop_len = 2
    
array1 = array
n_reps = ((1 - (array1.size-win_len)/hop_len % 1) * win_len)
print(n_reps)
array2 = np.concatenate((array,np.repeat(array[-1], n_reps)),0)

nrows = ((array2.size-win_len)//hop_len)+1
n = array.strides[0]
np.lib.stride_tricks.as_strided(array2, shape=(nrows,win_len), strides=(hop_len*n,n))

2.5


array([[ 0,  1,  2,  3,  4],
       [ 2,  3,  4,  5,  6],
       [ 4,  5,  6,  7,  8],
       [ 6,  7,  8,  9, 10],
       [ 8,  9, 10, 11, 12],
       [10, 11, 12, 13, 14],
       [12, 13, 14, 15, 16],
       [14, 15, 16, 17, 18],
       [16, 17, 18, 19, 20],
       [18, 19, 20, 21, 22],
       [20, 21, 22, 23, 24],
       [22, 23, 24, 25, 26],
       [24, 25, 26, 27, 28],
       [26, 27, 28, 29, 30],
       [28, 29, 30, 31, 32],
       [30, 31, 32, 33, 34],
       [32, 33, 34, 35, 36],
       [34, 35, 36, 37, 38],
       [36, 37, 38, 39, 40],
       [38, 39, 40, 41, 42],
       [40, 41, 42, 43, 44],
       [42, 43, 44, 45, 46],
       [44, 45, 46, 47, 48],
       [46, 47, 48, 49, 49]])

In [169]:
array2

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 13, 13, 13,
       13, 13, 13, 13, 13])

In [147]:
(array.size-win_len)/hop_len % 1

0.25

In [141]:
1 - 2.25 % 1

0.75

In [142]:
import math
math.ceil(0.75 * 5)

4

In [143]:
frame(x,5,4)

1
2


array([[0, 1, 2, 3, 4],
       [4, 5, 6, 7, 8]])

# Import data:

In [92]:
# DATA_PATH = Path("./../data")
# PAT_ID = Path("ET2")
# DATASET_NAME = Path("Pegboard_off")

# LFP_SOURCE = DATA_PATH / "preprocessed" / PAT_ID / DATASET_NAME.with_suffix(".h5")
# H5_SOURCE = DATA_PATH / "TF_and_df" / PAT_ID / DATASET_NAME.with_suffix(".h5")
# CSV_SOURCE = DATA_PATH / "TF_and_df" / PAT_ID / DATASET_NAME.with_suffix(".csv")

dataset1 = Patient(PatID.ET2).datasets[Task.Pegboard][Stim.OFF].load_1d()
dataset2 = Patient(PatID.ET2).datasets[Task.Pegboard][Stim.ON].load_1d()

In [94]:
dataset1.LFP.fs

204.8

In [93]:
dataset1.concatenate(dataset2)

Dataset: ET2 - Pegboard_OFF

                LFPs:

- Channel Names: None
- Shape: (5, 293555)
- End Time: 1433.369 s


                label: 

- Channel Names: ['label']
- Shape: (1, 2935545)
- End Time: 1433.371 s

### Read TF

In [12]:
h5_LFP = h5py.File(LFP_SOURCE, "r")

LFP = h5_LFP["LFP"][:]
label = h5_LFP["label"][:]

In [13]:
# LFP = (LFP - np.mean(LFP,-1,keepdims=True)) / np.std(LFP,-1,keepdims=True)

In [14]:
LFP[0].std()

19.820263840393924

In [15]:
h5 = h5py.File(H5_SOURCE, "r")
TF = h5["TF"][:]
# h5.close()

In [16]:
TF.shape

(5, 99, 1545386)

### Read df_data

In [17]:
df_data = make_label_df(label, 0.500)
df_data

,id_start,id_end,is_valid,label
0,0,1024,0,False
1,1024,2048,0,False
2,2048,3072,0,False
3,3072,4096,0,False
4,4096,5120,0,False
...,...,...,...,...
1504,1540096,1541120,1,False
1505,1541120,1542144,1,False
1506,1542144,1543168,1,False
1507,1543168,1544192,1,False


In [18]:
LFP.shape

(5, 1545386)

In [19]:
train_end = df_data[df_data["is_valid"] == 0]["id_end"].iloc[-1]


def get_norm_stats(LFP, train_end):
    return np.mean(LFP[:, :train_end], axis=-1), np.std(LFP[:, :train_end], axis=-1)


def norm_with_stats(LFP, stats):
    means, stds = stats[0], stats[1]
    return (LFP - means[:, np.newaxis]) / stds[:, np.newaxis]

In [20]:
stats = get_norm_stats(LFP, train_end)
LFP = norm_with_stats(LFP, stats)

In [21]:
fig, ax = plt.subplots()
ax.plot(LFP[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
means, stds = torch.tensor(xxx[0]), torch.tensor(xxx[1])

In [34]:
means

tensor([ 0.0013, -0.0005,  0.0027,  0.0005,  0.0005], dtype=torch.float64)

In [31]:
X = torch.tensor(LFP)
X.shape

torch.Size([5, 1545386])

In [38]:
(X - means[:, None]).shape

torch.Size([5, 1545386])

In [13]:
df_data["label"].value_counts(normalize=True)

True     0.559311
False    0.440689
Name: label, dtype: float64

In [14]:
df_data["label"].value_counts()

True     431
False    323
Name: label, dtype: int64

In [15]:
df_data

,id_start,id_end,is_valid,label
0,0,2048,0,False
1,2048,4096,0,False
2,4096,6144,0,False
3,6144,8192,0,False
4,8192,10240,0,False
...,...,...,...,...
749,1533952,1536000,1,False
750,1536000,1538048,1,False
751,1538048,1540096,1,False
752,1540096,1542144,1,False


# Baseline Classifier:

In [19]:
{"a": [33, 33], "b": [22, 22]}

{'a': [33, 33], 'b': [22, 22]}

In [30]:
bl_test = BLClassifier(LFP, df_data, extract_method="hilbert").get_feat_mats()
bl_test.X_train

,pow0_2_7Hz,pow0_8_12Hz,pow0_13_20Hz,pow0_21_30Hz,pow0_31_45Hz,pow0_46_60Hz,pow0_61_75Hz,pow0_76_100Hz,pow1_2_7Hz,pow1_8_12Hz,...,pow3_61_75Hz,pow3_76_100Hz,pow4_2_7Hz,pow4_8_12Hz,pow4_13_20Hz,pow4_21_30Hz,pow4_31_45Hz,pow4_46_60Hz,pow4_61_75Hz,pow4_76_100Hz
0,0.018029,0.001931,0.001806,0.003780,0.001647,0.003183,0.001329,0.003602,0.039928,0.002221,...,0.001638,0.004261,0.051060,0.007403,0.007965,0.002884,0.004780,0.036149,0.001416,0.003679
1,0.012849,0.001952,0.001930,0.006315,0.003187,0.003912,0.003067,0.004744,0.014793,0.007155,...,0.002790,0.003579,0.015974,0.008975,0.006728,0.007966,0.006054,0.047914,0.002421,0.003369
2,0.006197,0.001241,0.001016,0.003634,0.002056,0.001412,0.002927,0.004578,0.076811,0.006133,...,0.003530,0.004315,0.028349,0.004300,0.006908,0.007381,0.006079,0.039216,0.002323,0.003852
3,0.015469,0.007060,0.002954,0.002302,0.005152,0.001461,0.002192,0.002983,0.035249,0.009318,...,0.002287,0.002638,0.038692,0.012121,0.015480,0.006153,0.004910,0.030108,0.003984,0.003042
4,0.046608,0.011552,0.006550,0.006544,0.002491,0.002496,0.001838,0.002745,0.043684,0.004510,...,0.002371,0.003755,0.017758,0.005190,0.017386,0.010611,0.005967,0.051161,0.001964,0.004917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,0.047998,0.007361,0.004687,0.007183,0.004112,0.009520,0.003123,0.003262,0.117724,0.024391,...,0.003807,0.004121,0.075557,0.026540,0.008059,0.010834,0.009969,0.085988,0.004079,0.003584
1203,2.292164,1.430866,0.393183,0.085774,0.025296,0.013453,0.006761,0.007725,0.656414,0.732665,...,0.008685,0.004942,3.644939,3.614660,0.265406,0.042300,0.017047,0.092893,0.006541,0.006021
1204,4.600086,1.224434,0.043345,0.010965,0.004735,0.005767,0.003828,0.007901,4.743043,0.312119,...,0.003902,0.003834,2.249386,1.633039,0.076718,0.012283,0.011738,0.086949,0.001676,0.004269
1205,1.484608,0.068361,0.040103,0.010067,0.006563,0.008042,0.002271,0.003919,2.076224,0.099314,...,0.003459,0.005795,1.299161,0.063800,0.035326,0.010378,0.007960,0.083210,0.002453,0.002506


In [54]:
bl_cls = BLClassifier(LFP, df_data, extract_method="periodogram")
bl_cls.classify(method="SVM")

On Train: 
              precision    recall  f1-score   support

       False       0.81      0.98      0.89       544
        True       0.98      0.82      0.89       663

    accuracy                           0.89      1207
   macro avg       0.90      0.90      0.89      1207
weighted avg       0.91      0.89      0.89      1207

[[534  10]
 [122 541]]
AUC: 0.8988027903469079

On Valid:
              precision    recall  f1-score   support

       False       0.67      1.00      0.80       121
        True       1.00      0.67      0.81       181

    accuracy                           0.80       302
   macro avg       0.84      0.84      0.80       302
weighted avg       0.87      0.80      0.80       302

[[121   0]
 [ 59 122]]
AUC: 0.8370165745856354


[{'accuracy': {'train': 0.8906379453189727, 'valid': 0.804635761589404}},
 {'AUC': {'train': 0.8988027903469079, 'valid': 0.8370165745856354}},
 {'precision': {'train': 0.9818511796733213, 'valid': 1.0}},
 {'recall': {'train': 0.8159879336349924, 'valid': 0.6740331491712708}},
 {'f1': {'train': 0.8912685337726525, 'valid': 0.8052805280528054}}]

In [45]:
np.argmin(np.abs(3 - np.array([0, 2, 4, 6, 8])))

1

In [47]:
list(range(1))

[0]

In [37]:
out = bl_cls.classify_many(method="LDA")

In [42]:
out[1]

{'AUC': {'train': {'mean': 0.7132743362831858, 'std': 0.0013517922403999381},
  'valid': {'mean': 0.7434782608695654, 'std': 0.002173913043478271}}}

In [43]:
pd.DataFrame(out[0])

,accuracy
train,"{'mean': 0.6776119402985075, 'std': 0.001519925603633771}"
valid,"{'mean': 0.6874172185430464, 'std': 0.0026490066225165255}"


In [31]:
out[0][0]

{'accuracy': {'train': 0.6766169154228856, 'valid': 0.6821192052980133}}

In [48]:
bl_y_pred = bl_cls.y_pred.astype(float)

In [49]:
bl_cls.classify_lda()

On Train: 
              precision    recall  f1-score   support

       False       0.58      1.00      0.73       264
        True       1.00      0.43      0.60       339

    accuracy                           0.68       603
   macro avg       0.79      0.71      0.67       603
weighted avg       0.81      0.68      0.66       603

[[264   0]
 [194 145]]
AUC: 0.7138643067846607

On Valid:
              precision    recall  f1-score   support

       False       0.56      1.00      0.72        59
        True       1.00      0.49      0.66        92

    accuracy                           0.69       151
   macro avg       0.78      0.74      0.69       151
weighted avg       0.83      0.69      0.68       151

[[59  0]
 [47 45]]
AUC: 0.7445652173913043


(0.6782752902155887, 0.6887417218543046)

In [50]:
bl_lda_y_pred = bl_cls.lda_y_pred

In [46]:
y_pred = np.array(
    [
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
)

y_pred_2d = np.array(
    [
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.0,
        1.0,
        1.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
)

In [53]:
fig, ax = plt.subplots()
ax.plot(raw_label, label="Ground Truth")
ax.plot(
    df_data[df_data["is_valid"] == 1]["id_end"].values, y_pred * 0.8, label="1D-CNN"
)
ax.plot(
    df_data[df_data["is_valid"] == 1]["id_end"].values, y_pred_2d * 0.7, label="2D-CNN"
)

ax.plot(
    df_data[df_data["is_valid"] == 1]["id_end"].values, bl_y_pred * 0.5, label="SVM"
)
ax.plot(
    df_data[df_data["is_valid"] == 1]["id_end"].values, bl_lda_y_pred * 0.4, label="LDA"
)

ax.legend(loc="upper left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# TF = LFP

# Prepare learner:

## Dataloader prep

### Data stream:

In [21]:
def get_x(row):
    return torch.tensor(TF[:, :, row["id_start"] : row["id_end"]])


def get_y(row):
    return row["label"]


def splitter(df):
    train = df.index[df["is_valid"] == 0].tolist()
    valid = df.index[df["is_valid"] == 1].tolist()
    return train, valid

### Data transforms:

In [62]:
class Resizer(Transform):
    def __init__(self, target_size):
        self.target_size = target_size

    def encodes(self, X):
        if isinstance(X, TensorCategory):
            return X
        return torch.stack(
            [ToTensor()(ToPILImage()(x).resize(self.target_size)) for x in X]
        ).squeeze()


class LFPNormalizer(Transform):
    def __init__(self, means, stds):
        self.means, self.stds = means, stds

    def encodes(self, X):
        if isinstance(X, TensorCategory):
            return X
        xs = torch.unbind(X, 1)

        return torch.stack(
            [(x - self.means[ii]) / self.stds[ii] for ii, x in enumerate(xs)], 1
        )


class LFPNormalizer1d(Transform):
    def __init__(self, stats):
        self.means, self.stds = stats[0], stats[1]

    def encodes(self, X):
        if isinstance(X, TensorCategory):
            return X

        return

In [63]:
def LFP_block():
    return TransformBlock(
        item_tfms=[Resizer((160, 160)), IntToFloatTensor],
        batch_tfms=LFPNormalizer(
            [0.5, 0.5, 0.5, 0.5, 0.5], [0.098, 0.098, 0.098, 0.098, 0.098]
        ),
    )

In [64]:


def LFP_block1d():
    return TransformBlock(batch_tfms=(LFPNormalizer1d(get_norm_stats(LFP, train_end))))

### Dataloader:

In [65]:
dblock = DataBlock(
    blocks=(LFP_block, CategoryBlock),
    get_x=get_x,
    get_y=get_y,
    splitter=splitter,
)

In [39]:
# dblock.summary(df_data)

In [66]:
dls = dblock.dataloaders(df_data, bs=32)

TypeError: get_x() missing 1 required positional argument: 'row'

# Learner

In [36]:
resnet = myResNet(5, 2, [2, 3, 4, 3], 4)

In [37]:
resnet

ResNet(
  (0): ConvLayer(
    (0): Conv2d(5, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convs): Sequential(
        (0): ConvLayer(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [38]:
learn = Learner(
    dls,
    resnet,
    metrics=[accuracy],
    loss_func=F.cross_entropy,
    # cbs=[ActivationStats(with_hist=True)],
)

In [39]:
xb, yb = dls.one_batch()
yb.shape, xb.shape

(torch.Size([32]), torch.Size([32, 5, 160, 160]))

In [40]:
init_loss = learn.loss_func(learn.model(xb), yb)
init_loss

TensorCategory(0.6660, grad_fn=<AliasBackward>)

In [41]:
del xb, yb

In [34]:
learn.lr_find()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

SuggestedLRs(lr_min=0.0002511886414140463, lr_steep=0.04786301031708717)

In [42]:
learn.fit_one_cycle(5, 4 * 10e-4)

epoch,train_loss,valid_loss,accuracy,time
0,0.400885,1.561257,0.695364,05:07
1,0.286708,3.728475,0.397351,04:54
2,0.297099,0.147295,0.933775,04:51
3,0.278087,0.105526,0.966887,04:37


KeyboardInterrupt: 

In [43]:
preds_val = learn.get_preds()

In [44]:
labels_pred = torch.argmax(preds_val[0], -1).numpy()
labels_pred.shape

(151,)

In [45]:
y = df_data[df_data["is_valid"] == 1]["label"].astype(float).values

In [61]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.])

In [62]:
(labels_pred == y).mean()

0.9536423841059603

In [71]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import pandas as pd
import seaborn as sns

In [79]:
display_labels = ["Still", "Mvmt"]
cm = confusion_matrix(y, labels_pred)
cm_norm = confusion_matrix(y, labels_pred, normalize="true")
df_cm = pd.DataFrame(cm_norm, index=display_labels, columns=display_labels)

In [80]:
plt.figure()
sns.heatmap(df_cm, annot=True)
plt.show()
cm

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[57,  2],
       [ 5, 87]])

In [76]:
plt.show()